In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re
import time
import csv
import sys
import urllib.parse

In [3]:
def get_existing_product_titles(filename):
    existing_titles = set()
    with open(filename, mode='r', encoding='utf-8-sig') as file:
        reader = csv.DictReader(file)
        for row in reader:
            existing_titles.add(row['Product Title'])
    return existing_titles

def is_product_seen(product_title, existing_titles):
    if product_title in existing_titles:
        return True
    else:
        return False

from unidecode import unidecode

def convert_to_slug(text):
    text = unidecode(text)
    text = text.replace(' ', '-').lower()
    text = text.replace('&', 'va').lower()
    return text

# only for lazada
headers = ['Product Title', 'Brand', 'Price','Regular Price', 'Discount %','Image','Product Evaluation','Product_score_average', 'Detail', 'Thương hiệu', 'SKU', 'Dạng sản phẩm', 'Mẫu mã',
           'Phân loại thương hiệu', 'Công dụng', 'Ingredient', 'Loại da', 'Dạng sản phẩm', 'Định lượng',
           'Hạn sử dụng', 'Loại bảo hành', 'Xuất xứ', 'Loại đóng gói', 'Hướng dẫn sử dụng', 'Cỡ du lịch',
           'Số lô', 'Recommended Age','Delivery_Option_Instant','Thời gian bảo hành','Ngày sản xuất']

In [ ]:
# browser category_url base on category title.
url = "https://www.lazada.vn/#?"
category_base_url = "https://www.lazada.vn/{}/?{}"
response = requests.get(url)
browser = webdriver.Chrome()
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    category_wrapper = soup.find(id = "J_Categories")
    links = category_wrapper.find_all(class_="card-categories-li-content")

    for link in links:
        category_url = link.get("href")
        category_id = category_url.split("up_id=")[1].split("&")[0]
        category_title = link.find(class_="card-categories-name").text.strip()
        # input category title ---> chưa làm :v
        if(category_title == 'Dưỡng da & Serum') : 
            slug = convert_to_slug(category_title)
            category_url = category_base_url.format(slug, category_id)
            try:
                browser.get(category_url)
            except:
                print("Error navigating to category URL")
                continue
            
        else :
            break
else:
    print("Failed to access page")


In [5]:
browser = webdriver.Chrome()
base_url = "https://www.lazada.vn/kiehls1632706960/?q=All-Products&from=wangpu&langFlag=vi&page={}&pageTypeId=2"
# crawler_stock_test -> change file name in line 4 and line 6 to store your own data
with open('crawler_stock_test.csv', mode='a', newline='', encoding='utf-8-sig') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    existing_titles = get_existing_product_titles('crawler_stock_test.csv')    
    # just run at first time
    writer.writeheader() 
    current_page = 1
    
    url_with_page = base_url.format(current_page)
    while True:
        
        print("Page index:",current_page)

        try : 
            browser.get(url_with_page)
            pagination = browser.find_element(By.CLASS_NAME, 'ant-pagination')
            pagination_num = (pagination.find_elements(By.CLASS_NAME, "ant-pagination-item")[-1].text)
            print("Page Total:",pagination_num) 
        except Exception as e:
            print(e)

        if current_page > (int(pagination_num)) : 
            print("Reached the last page")
            break     
        time.sleep(5)

        products = browser.find_elements(By.CSS_SELECTOR, "[data-qa-locator='product-item']")
        
        listProductLink = []
        for product in products:
            outerHtml = product.get_attribute("outerHTML")
            productLink = re.search(r'href="(.*?)"', outerHtml).group(1)
            listProductLink.append(productLink)

        
        for productLink in listProductLink:
            print("DEBUG: " + productLink)

            try:
                browser.get("https://" + productLink)
            except:
                browser.get("https://www.lazada.vn/products" + productLink)

            time.sleep(2)  
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight*0.8);")
            time.sleep(8)  
            
            try:
                # basic information
                productTitle = browser.find_element(By.CLASS_NAME, "pdp-mod-product-badge-title").text
                if(is_product_seen(productTitle, existing_titles)):
                    print('Product is already in rows')
                    continue
                else : 

                    productBrand = browser.find_element(By.CLASS_NAME, "pdp-product-brand__brand-link").text
                    productEvaluation = browser.find_element(By.CLASS_NAME, "pdp-review-summary__link").text
                    productPrice = browser.find_element(By.CSS_SELECTOR, ".pdp-price_type_normal").text
                    productPrice = re.match(r'^[\d|\.|\,]+', productPrice).group()

                    try:
                        productRegularPrice_element = browser.find_element(By.CLASS_NAME, "pdp-price_type_deleted")
                        productRegularPrice = productRegularPrice_element.text
                    except NoSuchElementException:
                        productRegularPrice = 0

                    try:
                        DiscountPercent_element = browser.find_element(By.CLASS_NAME, "pdp-product-price__discount")
                        DiscountPercent = DiscountPercent_element.text
                    except NoSuchElementException:
                        DiscountPercent = 0
                    
                    productImage = browser.find_element(By.CLASS_NAME, "pdp-mod-common-image").get_attribute("src")
                    

                    expand_button = browser.find_element(By.CLASS_NAME, "pdp-view-more-btn")
                    ActionChains(browser).move_to_element(expand_button).perform()
                    time.sleep(4) 
                    expand_button.click()
                    time.sleep(4) 

                    # expand more information
                    detail_container = browser.find_element(By.CLASS_NAME, "detail-content")
                    span_tags = detail_container.find_elements(By.TAG_NAME, "span")
                    span_texts = []  
                    for span in span_tags:
                        span_text = span.text.strip()
                        span_texts.append(span_text)

                    detail_texts = [text for text in span_texts if text.strip()]

                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(6) 
                    
                    specification_container = browser.find_element(By.CLASS_NAME, "pdp-mod-specification")
                    li_tags = specification_container.find_elements(By.TAG_NAME, "li")      

                    specifications = {}
                    for li_tag in li_tags:
                        title_element = li_tag.find_element(By.CLASS_NAME, "key-title")
                        value_element = li_tag.find_element(By.CLASS_NAME, "key-value")

                        title = title_element.text.strip()
                        value = value_element.text.strip()
                        if title in headers:
                            specifications[title] = value
                        else :
                            headers.append(title)
                            specifications[title] = value

                    Product_score_average = browser.find_element(By.CLASS_NAME, "score-average").text
                    
                    print(current_page)
                    row_data = {
                        'Product Title': productTitle,
                        'Brand': productBrand,
                        'Price': productPrice,
                        'Regular Price':productRegularPrice,
                        'Discount %': DiscountPercent,
                        'Image': productImage,
                        'Product Evaluation' : productEvaluation ,
                        'Product_score_average': Product_score_average,
                        'Detail' : detail_texts,
                        **specifications 
                    }
                    writer.writerow(row_data)
                    print('done')
                    time.sleep(3)
            except NoSuchElementException:
                print("Error: Product information not found")
        current_page += 1
        base_url="https://www.lazada.vn/kiehls1632706960/?q=All-Products&from=wangpu&langFlag=vi&page={}&pageTypeId=2"
        time.sleep(2)
browser.quit()

Page index: 1
Page Total: 1
DEBUG: //www.lazada.vn/products/kem-duong-am-kiehls-ultra-facial-cream-50ml-i2060053256.html
1
done
DEBUG: //www.lazada.vn/products/deal-tinh-chat-serum-lam-sang-da-mo-tham-mun-kiehls-clearly-corrective-dark-spot-solution-30ml-i2060002390.html


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00007FF7CF3D1502+60802]
	(No symbol) [0x00007FF7CF34AC02]
	(No symbol) [0x00007FF7CF207CE4]
	(No symbol) [0x00007FF7CF1DDFDF]
	(No symbol) [0x00007FF7CF281E57]
	(No symbol) [0x00007FF7CF2998D1]
	(No symbol) [0x00007FF7CF27A923]
	(No symbol) [0x00007FF7CF248FEC]
	(No symbol) [0x00007FF7CF249C21]
	GetHandleVerifier [0x00007FF7CF6D411D+3217821]
	GetHandleVerifier [0x00007FF7CF7160B7+3488055]
	GetHandleVerifier [0x00007FF7CF70F03F+3459263]
	GetHandleVerifier [0x00007FF7CF48B846+823494]
	(No symbol) [0x00007FF7CF355F9F]
	(No symbol) [0x00007FF7CF350EC4]
	(No symbol) [0x00007FF7CF351052]
	(No symbol) [0x00007FF7CF3418A4]
	BaseThreadInitThunk [0x00007FFD81A9257D+29]
	RtlUserThreadStart [0x00007FFD8298AA48+40]
